In [ ]:
# imports
import cv2 as cv
import numpy as np
import time
from matplotlib import pyplot as plt
import glob
from prettytable import PrettyTable

# Time Measurement

In [32]:
time_measurements = {}

def start_time_measurement(eventName):
    add = False
    if eventName not in time_measurements:
        time_measurements[eventName] = {
            "name": eventName,
            "start": [],
            "end": [],
            "count": 0
        }
        add = True
    elif len(time_measurements[eventName]["start"]) > len(time_measurements[eventName]["end"]):
        print(f"Time measure error: Event '{eventName}' not finished before reassignment!")
    else:
       add = True
    
    #start measurement as late as possible
    if add:
        time_measurements[eventName]["start"].append(time.process_time_ns())

def end_time_measurement(eventName):
    #end measurement as fast as possible
    temp_time = time.process_time_ns()
    if eventName not in time_measurements:
        print(f"Time measure error: Event '{eventName}' not defined!")
    elif len(time_measurements[eventName]["end"]) >= len(time_measurements[eventName]["start"]):
        print(f"Time measure error: Event '{eventName}' not started before reassignment!")
    else:
        time_measurements[eventName]["end"].append(temp_time)
        time_measurements[eventName]["count"] += 1

def analyse_time_measurements():
    time_measurements_table = PrettyTable(["Name", "Avg. [ms]", "Min. [ms]", "Max. [ms]", "Occurrences [compl.]"])
    time_measurements_table.align["Name"] = "l"
    time_measurements_table.align["Avg. [ms]"] = "r"
    time_measurements_table.align["Min. [ms]"] = "r"
    time_measurements_table.align["Max. [ms]"] = "r"
    time_measurements_table.align["Occurrences [compl.]"] = "r"
    for key, event in time_measurements.items():
        timings = []
        if len(event["start"]) != len(event["end"]):
            print(f"Time measure error: Event '{key}' has different amounts of values for start and end times!")
        else:
            #exclude 0 values
            for i in range(len(event["start"])):
                timing = (event["end"][i] - event["start"][i])
                if timing >= 0:
                    timing = timing / (1000 * 1000) #convert from ns to ms
                    timings.append(timing)

            event["min"] = min(timings) 
            event["max"] = max(timings)
            event["avg"] = sum(timings) / len(event["start"])

            time_measurements_table.add_row([key, '{0:.2f}'.format(event["avg"]), '{0:.2f}'.format(event["min"]), '{0:.2f}'.format(event["max"]), event["count"]])
    print(time_measurements_table)

# Kamerakalibrierung

In [ ]:
# size of chessboard, minimum error with (7, 6), but there were severe artefacts at the borders (see error calculation at the end)
x, y = 9, 6

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((y * x, 3), np.float32)
objp[:, :2] = np.mgrid[0:x, 0:y].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.


images = glob.glob("./img/Udacity/calib/*.jpg")
for i, fname in enumerate(images):
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (x, y), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(
            gray, corners, (11, 11), (-1, -1), criteria
        )  # improve accuracy of corners
        imgpoints.append(corners)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))


def undistort_image(img):
    h, w = img.shape[:2]

    img_undist = cv.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    return img_undist[y : y + h, x : x + w]


# ~50% faster than undistort_image()
def undistort_image_remap(img):
    h, w = img.shape[:2]
    mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
    dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
    # crop the image
    x, y, w, h = roi
    return dst[y : y + h, x : x + w]

# Perspektivtransformation

Mögliche Performance-verbesserung:

- Bild nach warp verkleinern, da ein großteil des Bildes aus wenigen Pixeln entsteht ->


In [65]:
# udacity images
src_udacity = np.float32([[191, 628], [531, 404], [1021, 628], [681, 404]])
dst_udacity = np.float32([[150, 720], [150, 10], [1000, 720], [1000, 10]])
M = cv.getPerspectiveTransform(src_udacity, dst_udacity)


def warp_image_udacity(img):
    image = cv.warpPerspective(img, M, (img.shape[1], img.shape[0]))
    image = cv.resize(
        image, (int(img.shape[1] / 2), int(img.shape[0] / 2))
    )  # resize to half size
    return image

# Main


In [35]:
start_time_measurement("Application")
# Open video file
video_file = "project"
capture = cv.VideoCapture("./img/Udacity/" + video_file + "_video.mp4")

# Check if camera opened successfully
if capture.isOpened() == False:
    print("Error opening video stream or file")

# Start timer for fps counter
start_timer = time.time() - 0.01
frame_count = -1

# Read every frame
while capture.isOpened():
    ret, frame = capture.read()

    # Check if there is another frame
    if frame is None:
        break
    orig = frame.copy()

    # Calculate Frame rate
    frame_count += 1
    ellapsed_time = time.time() - start_timer
    frame_rate = frame_count / ellapsed_time

    if ret == True:

        frame = undistort_image_remap(frame)
        frame = warp_image_udacity(frame)
        
        
        #treshhold for image
        thresh = 160
        ret, frame = cv.threshold(frame, thresh=thresh, maxval=255, type=cv.THRESH_BINARY)

        #deblur image
        frame = cv.medianBlur(frame,5)

        # Schärfungsfilter
        kernel2 = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]],np.float32)
        frame = cv.filter2D(frame,-1,kernel2)

        # Add frame rate to video
        cv.putText(frame,"FPS: " + str(round(frame_rate)),(0, 25),cv.FONT_HERSHEY_SIMPLEX,1,(255, 255, 0),2,cv.LINE_AA,)
        cv.imshow("Frame", frame)

        # Close video with letter 'q'
        if cv.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# When everything done, release the video capture object
capture.release()
cv.destroyAllWindows()

end_time_measurement("Application")
analyse_time_measurements()

+-----------------+-----------+-----------+-----------+----------------------+
| Name            | Avg. [ms] | Min. [ms] | Max. [ms] | Occurrences [compl.] |
+-----------------+-----------+-----------+-----------+----------------------+
| Application     |  32812.50 |  32812.50 |  32812.50 |                    1 |
| Frame undistort |     19.13 |      0.00 |    156.25 |                 1199 |
| Frame warp      |      5.80 |      0.00 |    171.88 |                 1199 |
+-----------------+-----------+-----------+-----------+----------------------+
